## Baseline

Now that we have our threshold (refer to last line in our EDA Part2) and our definitions we will begin modelling our data.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import polars as pl
import seaborn as sns
from sklearn.model_selection import train_test_split
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import matplotlib.pyplot as plt
import tqdm 
import os
from surprise import Dataset, Reader, accuracy
from surprise.model_selection import cross_validate, KFold, GridSearchCV
from surprise import NormalPredictor, BaselineOnly, KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline, SVD, NMF, SlopeOne, CoClustering    
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/user-ratings/user_ratings.csv
/kaggle/input/games-after-2010/games_after_2010.csv
/kaggle/input/df-for-model/df_for_modelling.csv


In [ ]:
with open('..data/data_for_model/data_for_model.pkl', 'rb') as f:
    # Load the contents of the file using pickle.load()
    data_for_model = pickle.load(f)

In [33]:
algos = [('Baseline',BaselineOnly()),('Normal_predictor',NormalPredictor())]

In [36]:
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(data_for_model[['Username','BGGId', 'Rating']],reader)

Using the recommended metrics by scikit suprise, we will be using recall@k and precision@k to compute the performance of our model. Precision is essentially how many of our predictions our model gave us that we deemed to be relevant to our user in our top 'k' recommendations. Recall is essentially how many of the predictions our model gave out of all relevant items in our top 'k' recommendations, essentially the variability of our recommendation when looking at all relevant recommendations as a whole.

In our specific use case our threshold would be 7.5 and our k will be set to 10.

Refer to the following [link](https://medium.com/@m_n_malaeb/recall-and-precision-at-k-for-recommender-systems-618483226c54) for a more detailed explanation.

In [1]:
'''https://surprise.readthedocs.io/en/stable/FAQ.html#how-to-compute-precision-k-and-recall-k'''
from collections import defaultdict

from surprise import Dataset, SVD
from surprise.model_selection import KFold


def precision_recall_at_k(predictions, k=10, threshold=5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls



We used a 5 fold cross validation to train and test our model to determine the metrics we want including our loss (RMSE).

In [42]:
# # # # #############really need to sort this out ####################
precs_list = []
recalls_list = []
kf = KFold(n_splits=5,random_state=42)
result_all = []
for var_name,model in algos:
    temp_prec_list = []
    temp_recall_list = []
    for trainset, testset in kf.split(data):
        temp_result= []
        model.fit(trainset)
        predictions = model.test(testset)
        precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=7.5)
        result = cross_validate(model,data,measures=['RMSE','MAE'],cv=5,verbose=True)
        temp_result.append(result['test_rmse'].mean())
        result_all.append((var_name,temp_result))
    # Precision and recall can then be averaged over all users
        temp_prec_list.append(sum(prec for prec in precisions.values()) / len(precisions))
        temp_recall_list.append(sum(rec for rec in recalls.values()) / len(recalls))
    precs_list.append((var_name,np.mean(temp_prec_list)))
    recalls_list.append((var_name,np.mean(temp_recall_list)))
    

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0196  1.0223  1.0226  1.0196  1.0187  1.0206  0.0016  
MAE (testset)     0.7591  0.7600  0.7624  0.7593  0.7600  0.7602  0.0012  
Fit time          3.46    3.81    4.22    3.81    3.83    3.83    0.24    
Test time         1.77    1.73    2.30    1.76    1.72    1.86    0.22    
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0193  1.0246  1.0198  1.0203  1.

In [43]:
recalls_list

[('Baseline', 0.3679935196001593), ('Normal_predictor', 0.31420880432844184)]

In [44]:
precs_list

[('Baseline', 0.6861302490644018), ('Normal_predictor', 0.4658070331655703)]

In [45]:
result_all

[('Baseline', [1.0205603642904433]),
 ('Baseline', [1.0204669150813224]),
 ('Baseline', [1.0205703614676553]),
 ('Baseline', [1.0206715940241646]),
 ('Baseline', [1.0205877974894502]),
 ('Normal_predictor', [1.851355658968064]),
 ('Normal_predictor', [1.854540711200352]),
 ('Normal_predictor', [1.8519912733150865]),
 ('Normal_predictor', [1.8551386751747798]),
 ('Normal_predictor', [1.8539058727766673])]

In [46]:
import pickle
def pickle_lists(list_of_lists):
    for i, sublist in enumerate(list_of_lists):
        with open(f"../data/pickled_baseline/{i}.pickle", "wb") as f:
            pickle.dump(sublist, f)

In [47]:
pickle_lists([recalls_list,precs_list,result_all])

### Conclusions

Due to computational constraints(long run time and memory errors) our modelling and metrics will be done in seperate notebooks and the data was pickled for further examination in model_final notebook.